In [9]:
import pandas as pd

In [10]:
import gensim
import gensim.downloader as download_api
russian_model = download_api.load('word2vec-ruscorpora-300')

In [11]:
len(russian_model)

184973

In [690]:
russian_model.most_similar(positive=['радость_NOUN'])

[('радостный_ADJ', 0.7136526107788086),
 ('восторг_NOUN', 0.677175760269165),
 ('счастие_NOUN', 0.6690915822982788),
 ('ликование_NOUN', 0.6549766063690186),
 ('печаль_NOUN', 0.645858645439148),
 ('огорчение_NOUN', 0.6326740384101868),
 ('горесть_NOUN', 0.6323299407958984),
 ('веселие_NOUN', 0.6247930526733398),
 ('радоваться_VERB', 0.6218827366828918),
 ('удовольствие_NOUN', 0.612714409828186)]

# Считаем процент

In [480]:
import os
import glob
import fileinput
import csv

In [268]:
path = "D:/patients"
pattern = '*.txt'

glob_path = os.path.join(path, pattern)
list_files = glob.glob(glob_path)
new_file = 'new_patient.txt'

if list_files:
    with fileinput.FileInput(files=list_files) as fr, open(new_file, 'w') as fw:
        for line in fr:
            fw.write(line)

In [269]:
path = "D:/control group"
pattern = '*.txt'

glob_path = os.path.join(path, pattern)
list_files = glob.glob(glob_path)
new_file = 'new_control.txt'

if list_files:
    with fileinput.FileInput(files=list_files) as fr, open(new_file, 'w') as fw:
        for line in fr:
            fw.write(line)

In [692]:
with open("C:/Users/Ольга/new_patient.txt", encoding='utf-8') as f:
    patients = f.read()
with open("C:/Users/Ольга/new_control.txt", encoding='utf-8') as f:
    control = f.read()

In [567]:
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
stopwords_ru = stopwords.words("russian")

In [693]:
def lem_and_tok(text):
    morph = MorphAnalyzer()
    tokens = []
    words = word_tokenize(text, language="russian")
    for word in words:
        if word not in stopwords_ru:
            word = word.strip()
            word = morph.normal_forms(word)[0]
            tokens.append(word)
    return tokens

In [694]:
patients_ready = lem_and_tok(patients)
len(patients_ready)

1767

In [695]:
control_ready = lem_and_tok(control)
len(control_ready)

8357

In [18]:
with open("C:/Users/Ольга/Desktop/радость.txt", encoding='utf-8') as f:
    good = f.read()
    good = good.split(sep=',')
    
print(len(good))

195


In [19]:
with open("C:/Users/Ольга/Desktop/печаль.txt", encoding='utf-8') as f:
    bad = f.read()
    bad = bad.split(sep=',')
print(len(bad))

160


In [699]:
def calculate(file):
    percent_good = []
    percent_bad = []
    for elem in file:
        if elem in good:
            percent_good.append(elem)
        elif elem in bad:
            percent_bad.append(elem)

    per_cent_good = len(percent_good) / len(file) * 100
    per_cent_bad = len(percent_bad) / len(file) * 100
    return (f"""percent of positive vocabulary is {round(per_cent_good)}%, amount: {len(percent_good)}, 
           percent of negative vocabulary is {round(per_cent_bad)}%, amount: {len(percent_bad)}""")

In [700]:
print('Patients:', calculate(patients_ready))

Patients: percent of positive vocabulary is 4%, amount: 64, 
           percent of negative vocabulary is 2%, amount: 27


In [701]:
print('Control group:', calculate(control_ready))

Control group: percent of positive vocabulary is 4%, amount: 337, 
           percent of negative vocabulary is 0%, amount: 22


# FOR EVERY PERSON

In [736]:
path = "D:/patients"

glob_path = os.path.join(path, pattern)
list_files = glob.glob(glob_path)
new_file = 'patient_model.txt'

if list_files:
    with fileinput.FileInput(files=list_files) as fr, open(new_file, 'w') as fw:
        for line in fr:
            fw.write(line)
            fw.write(",")

In [12]:
with open("C:/Users/Ольга/patient_model.txt", encoding='utf-8') as f:
    patients_m = f.read()
    patients_m = patients_m.split(sep=',')

In [720]:
path = "D:/control group"

glob_path = os.path.join(path, pattern)
list_files = glob.glob(glob_path)
new_file = 'control_model.txt'

if list_files:
    with fileinput.FileInput(files=list_files) as fr, open(new_file, 'w') as fw:
        for line in fr:
            fw.write(line)
            fw.write(",")

In [13]:
with open("C:/Users/Ольга/control_model.txt", encoding='utf-8') as f:
    control_m = f.read()
    control_m = control_m.split(sep=',')

In [16]:
def calculate_1(file):
    percent_good = []
    percent_bad = []
    for elem in file:
        if elem in good:
            percent_good.append(elem)
        elif elem in bad:
            percent_bad.append(elem)

    per_cent_good = len(percent_good) / len(file) * 100
    per_cent_bad = len(percent_bad) / len(file) * 100
    return round(per_cent_good, 2), round(per_cent_bad, 2)

In [20]:
pos_patients = []
neg_patients = []
for elem in patients_m:
    a, b = calculate_1(elem.split(sep=' '))
    pos_patients.append(a)
    neg_patients.append(b)

In [21]:
pos_conrol = []
neg_conrol = []
for elem in control_m:
    a, b = calculate_1(elem.split(sep=' '))
    pos_conrol.append(a)
    neg_conrol.append(b)

# DATA

In [22]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [23]:
data = pd.read_csv("C:/Users/Ольга/Desktop/contol_group  - errors.csv")

data['positive voc'] = pos_patients + pos_conrol
data['negative voc'] = neg_patients + neg_conrol

diag = []
for elem in data['group']:
    diag.append(1) if elem == 'control' else diag.append(0)

data = data.drop(['ID', 'group', 'group1', 'diagnosis'], axis=1)

data['group'] = diag

data = pd.get_dummies(data, drop_first=True)
data.head()

data.head()

,age,years of education,errors,...=,...*,positive voc,negative voc,group,sex_male,education_higher.unfinished,education_secondary,education_vocational,education_vocational.unfinished,education_ученая степень
0,36,17,0,0,0,3.03,0.0,1,0,0,0,0,0,0
1,27,18,0,0,0,0.00,0.0,1,0,0,0,0,0,0
2,20,14,0,0,0,0.00,0.0,1,1,0,0,0,0,0
3,21,14,1,0,1,9.09,0.0,1,0,0,0,0,0,0
4,25,18,0,0,0,0.00,0.0,1,1,0,0,0,0,0


In [24]:
data.corr()

,age,years of education,errors,...=,...*,positive voc,negative voc,group,sex_male,education_higher.unfinished,education_secondary,education_vocational,education_vocational.unfinished,education_ученая степень
age,1.000000,0.401527,-0.010060,0.031253,-0.079723,-0.005431,-0.138017,0.217570,0.096716,-0.262900,-0.175465,0.035411,-0.206103,0.097678
years of education,0.401527,1.000000,-0.070644,-0.044896,-0.082554,0.075796,-0.080558,0.490711,-0.005892,-0.159539,-0.617406,-0.145793,-0.263816,0.306703
errors,-0.010060,-0.070644,1.000000,0.911591,0.670172,0.082198,-0.002785,-0.380058,0.044347,-0.030803,0.012980,0.001942,0.167003,0.032233
...=,0.031253,-0.044896,0.911591,1.000000,0.305803,0.055877,-0.006944,-0.412925,-0.012300,-0.059980,-0.001514,0.007473,0.134517,0.021371
...*,-0.079723,-0.082554,0.670172,0.305803,1.000000,0.089487,0.006086,-0.134702,0.124913,0.036950,0.032794,-0.008995,0.143917,0.036067
positive voc,-0.005431,0.075796,0.082198,0.055877,0.089487,1.000000,-0.030746,0.057569,-0.040067,0.033504,-0.088469,0.076512,-0.027768,0.032694
negative voc,-0.138017,-0.080558,-0.002785,-0.006944,0.006086,-0.030746,1.000000,0.082948,-0.142237,-0.037327,-0.112333,-0.066566,0.104408,-0.068011
group,0.217570,0.490711,-0.380058,-0.412925,-0.134702,0.057569,0.082948,1.000000,0.105294,0.073511,-0.310517,-0.073771,-0.387298,0.120386
sex_male,0.096716,-0.005892,0.044347,-0.012300,0.124913,-0.040067,-0.142237,0.105294,1.000000,0.182002,0.156269,-0.115728,-0.115728,0.158277
education_higher.unfinished,-0.262900,-0.159539,-0.030803,-0.059980,0.036950,0.033504,-0.037327,0.073511,0.182002,1.000000,-0.200805,-0.132186,-0.132186,-0.098611


# TRAIN/TEST

In [25]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn. model_selection import LeaveOneOut
from sklearn. model_selection import cross_val_score
from numpy import mean

In [37]:
X = data.drop('group', axis=1) 
y = data['group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [38]:
print(len(X_train), len(y_val))

71 24


# LOG REGRESSION

In [28]:
cv = LeaveOneOut()

In [29]:
log_reg = LogisticRegression(max_iter=10000)

In [30]:
precision = cross_val_score(log_reg, X, y, scoring='precision',
 cv=cv, n_jobs=-1)

recall = cross_val_score(log_reg, X, y, scoring='recall',
 cv=cv, n_jobs=-1)

accuracy = cross_val_score(log_reg, X, y, scoring='accuracy',
 cv=cv, n_jobs=-1)

print(f'Precision: {mean(precision)}, recall: {mean(recall)}, accuracy: {mean(accuracy)}')

Precision: 0.6470588235294118, recall: 0.6470588235294118, accuracy: 0.8403361344537815


# RandomForestClassifier

In [31]:
rf = RandomForestClassifier()
cv1 = LeaveOneOut()

In [39]:
hyperparams = {
    'n_estimators': [5, 25, 50, 100, 150, 200],
    'max_depth': [6, 12, 24, 48, 96, None]
}

cross_val = GridSearchCV(rf, hyperparams, cv=5)
cross_val.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [6, 12, 24, 48, 96, None],
                         'n_estimators': [5, 25, 50, 100, 150, 200]})

In [40]:
def results(results):
    print(f'Optimal Hyperparams: {results.best_params_}')
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']

    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print(f'Mean {round(mean, 3)}, Standart Deviation {round(std * 2, 3)} Hyperparameters {params}')

In [41]:
results(cross_val)

Optimal Hyperparams: {'max_depth': 6, 'n_estimators': 100}
Mean 0.789, Standart Deviation 0.181 Hyperparameters {'max_depth': 6, 'n_estimators': 5}
Mean 0.803, Standart Deviation 0.228 Hyperparameters {'max_depth': 6, 'n_estimators': 25}
Mean 0.831, Standart Deviation 0.192 Hyperparameters {'max_depth': 6, 'n_estimators': 50}
Mean 0.859, Standart Deviation 0.239 Hyperparameters {'max_depth': 6, 'n_estimators': 100}
Mean 0.83, Standart Deviation 0.233 Hyperparameters {'max_depth': 6, 'n_estimators': 150}
Mean 0.83, Standart Deviation 0.233 Hyperparameters {'max_depth': 6, 'n_estimators': 200}
Mean 0.818, Standart Deviation 0.243 Hyperparameters {'max_depth': 12, 'n_estimators': 5}
Mean 0.803, Standart Deviation 0.245 Hyperparameters {'max_depth': 12, 'n_estimators': 25}
Mean 0.845, Standart Deviation 0.246 Hyperparameters {'max_depth': 12, 'n_estimators': 50}
Mean 0.816, Standart Deviation 0.234 Hyperparameters {'max_depth': 12, 'n_estimators': 100}
Mean 0.816, Standart Deviation 0.234 

In [42]:
rf1 = RandomForestClassifier(n_estimators=100, max_depth=6)

rf2 = RandomForestClassifier(n_estimators=50, max_depth=12)

rf3 = RandomForestClassifier(n_estimators=50, max_depth=None)

In [43]:
cv1 = LeaveOneOut()

In [44]:
r = [rf1, rf2, rf3]

for mdl in r:
    precision = cross_val_score(mdl, X, y, scoring='precision', cv=cv1, n_jobs=-1)
    recall = cross_val_score(mdl, X, y, scoring='recall', cv=cv1, n_jobs=-1)
    accuracy = cross_val_score(mdl, X, y, scoring='accuracy', cv=cv1, n_jobs=-1)
    print(f'Precision: {mean(precision)}, recall: {mean(recall)}, accuracy: {mean(accuracy)}')

Precision: 0.6554621848739496, recall: 0.6386554621848739, accuracy: 0.8319327731092437
Precision: 0.6470588235294118, recall: 0.6470588235294118, accuracy: 0.8487394957983193
Precision: 0.6386554621848739, recall: 0.6386554621848739, accuracy: 0.8319327731092437


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
>>>
X, y = make_classification(n_samples=1000, n_classes=3,
                           n_informative=4, weights=[0.2, 0.3, 0.5],
                           random_state=0)
clf = BalancedRandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
BalancedRandomForestClassifier(...)
print(clf.feature_importances_)
[...]
print(clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

# BalancedRandomForestClassifier

In [45]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [46]:
cv2 = LeaveOneOut()

In [47]:
brf1 = RandomForestClassifier(n_estimators=100, max_depth=6)

brf2 = RandomForestClassifier(n_estimators=50, max_depth=12)

brf3 = RandomForestClassifier(n_estimators=50, max_depth=None)

In [49]:
br = [brf1, brf2, brf3]

for mdl in br:
    precision = cross_val_score(mdl, X, y, scoring='precision', cv=cv2, n_jobs=-1)
    recall = cross_val_score(mdl, X, y, scoring='recall', cv=cv2, n_jobs=-1)
    accuracy = cross_val_score(mdl, X, y, scoring='accuracy', cv=cv2, n_jobs=-1)
    print(f'Precision: {mean(precision)}, recall: {mean(recall)}, accuracy: {mean(accuracy)}')

Precision: 0.6638655462184874, recall: 0.6638655462184874, accuracy: 0.8403361344537815
Precision: 0.6386554621848739, recall: 0.6386554621848739, accuracy: 0.8403361344537815
Precision: 0.6302521008403361, recall: 0.6386554621848739, accuracy: 0.8319327731092437
